In [18]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,MaxPooling2D,Input,Dense,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [19]:
train_datagen=ImageDataGenerator(rescale=1./255,validation_split=0.2)

In [37]:
batchsize=8

In [58]:
train_data=train_datagen.flow_from_directory(r'D:\Project\train\Train',target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training')

validation_data=train_datagen.flow_from_directory(r'D:\Project\train\Train',target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='validation')

Found 2880 images belonging to 2 classes.
Found 720 images belonging to 2 classes.


In [59]:
test_datagen=ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(r'D:\Project\train\Test',target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 400 images belonging to 2 classes.


In [57]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [31]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv2d_472 (Conv2D)            (None, 39, 39, 32)   864         ['input_12[0][0]']               
                                                                                                  
 batch_normalization_470 (Batch  (None, 39, 39, 32)  96          ['conv2d_472[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_470 (Activation)    (None, 39, 39, 32)   0           ['batch_normalization_470[0][

In [32]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [50]:
checkpoint = ModelCheckpoint(r'D:\Project\model\model.h5',monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor='val_loss', patience=7,verbose=3,restore_best_weights=True)

learningrate = ReduceLROnPlateau(monitor='val_loss',patience=7,verbose=3)

callbacks = [checkpoint,earlystop,learningrate]

In [51]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=5)

Epoch 1/5


C:\Users\vevin\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


359/360 [============================>.] - ETA: 0s - loss: 2.3150e-05 - accuracy: 1.0000
Epoch 1: val_loss improved from inf to 0.79624, saving model to D:\Project\model\model.h5
360/360 [==============================] - 22s 53ms/step - loss: 2.3085e-05 - accuracy: 1.0000 - val_loss: 0.7962 - val_accuracy: 0.9528 - lr: 0.0010
Epoch 2/5
360/360 [==============================] - ETA: 0s - loss: 8.4128e-07 - accuracy: 1.0000
Epoch 2: val_loss did not improve from 0.79624
360/360 [==============================] - 19s 54ms/step - loss: 8.4128e-07 - accuracy: 1.0000 - val_loss: 1.1670 - val_accuracy: 0.9319 - lr: 0.0010
Epoch 3/5
359/360 [============================>.] - ETA: 0s - loss: 1.5489e-05 - accuracy: 1.0000
Epoch 3: val_loss did not improve from 0.79624
360/360 [==============================] - 22s 62ms/step - loss: 1.5446e-05 - accuracy: 1.0000 - val_loss: 0.9899 - val_accuracy: 0.9542 - lr: 0.0010
Epoch 4/5
359/360 [============================>.] - ETA: 0s - loss: 0.0122 - a

In [48]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

C:\Users\vevin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


3.285478555881127e-07
1.0
